<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_05_17_rivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wstęp

W tym skrypcie zastosujemy metodę opartą na pracy [Rivers, D. (2007). Sampling for web surveys, Proceedings of the Survey Research Methods
Section of the American Statistical Association, pp. 1–26.](https://static.texastribune.org/media/documents/Rivers_matching4.pdf). 

W tym ćwiczeniu porównamy dwa podejścia:

1. wykorzystując zmienne $\mathbf{X}$ do łączenia
2. wykorzystując wartości przewidywane $\hat{y}_i$ na podstawie modelu regresji logistycznej

Poszukiwania sąsiadów dokonamy wykorzystując metodę K-D tree dla przybliżonych sąsiadów oprogramowaną w pakiecie `RANN` (funkcja `nn2`), która opiera się na bibliotece [ANN w C++](http://www.cs.umd.edu/~mount/ANN/).

# Pakiety i dane

Instalujemy pakiet

In [1]:
install.packages("RANN")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Ładujemy pakiety

In [ ]:
library(tidyverse) 
library(RANN) 

Pobieramy dane z zajęć

In [4]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")

In [5]:
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE)
head(dane)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA


## Metoda Rivers'a -- łączenie na podstawie zmiennych X

Na tę chwilę założymy, że zbiory są rozłączne!

In [10]:
proba_nielosowa <- dane %>% filter(is.na(id_popyt))
proba_losowa <- dane %>% filter(!is.na(id_popyt))

Tworzymy macierz X_A (próba nielosowa) i X_B (próba losowa)

In [12]:
X_A <- model.matrix(~ -1 + klasa_pr + sekc_pkd + woj + zawod_kod2, data = proba_nielosowa)
X_B <- model.matrix(~ -1 + klasa_pr + sekc_pkd + woj + zawod_kod2, data = proba_losowa)
head(X_B)

,klasa_prD,klasa_prM,klasa_prS,sekc_pkdD.E,sekc_pkdF,sekc_pkdG,sekc_pkdH,sekc_pkdI,sekc_pkdJ,sekc_pkdK.L,sekc_pkdM,sekc_pkdN,sekc_pkdO,sekc_pkdP,sekc_pkdQ,sekc_pkdR.S,woj,zawod_kod2
1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14,1
2,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14,2
3,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,24,2
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,14,2
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,14,2
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,22,2


In [13]:
dim(X_A)
dim(X_B)

[1] 10972    18

[1] 9974   18

W zbiorze A mamy 10972 obserwacje, a zbiorze B 9974. Oznacza to, że musimy znaleźć 9974 jednostek ze zbioru A dla jednostek ze zbioru B (mogą się powtarzać).

In [14]:
sasiedzi <- nn2(data = X_A, query = X_B, k = 1)
str(sasiedzi,1)

List of 2
 $ nn.idx  : int [1:9974, 1] 1293 1608 603 183 183 9967 5892 10352 6687 9471 ...
 $ nn.dists: num [1:9974, 1] 1 0 0 0 0 1 1 0 0 1 ...


Funkcja `nn2` zwraca listę składającą się z dwóch wektorów:

+ `nn.indx` -- indeksy (numery wierszy) ze zbioru `X_A`,
+ `nn.dists` -- odległości między sąsiadami ze zbiorów `X_A` i `X_B` 

Wyznaczamy teraz estymator $\hat{\theta}_{M1}$ czyli chcemy poznać odsetek podmiotów poszukujących pracowników na jedną zmianę.

In [22]:
indeksy <- sasiedzi$nn.idx[,1]
y <- proba_nielosowa$jedna_zmiana[indeksy]
naiwny <- weighted.mean(proba_nielosowa$jedna_zmiana, proba_nielosowa$wolne_miejsca_cbop)
theta_m1 <- weighted.mean(y, proba_losowa$waga*proba_losowa$wolne_miejsca)
print(c(naiwny = naiwny, theta_m1 = theta_m1))

   naiwny  theta_m1 
0.5204615 0.6601766 


## Metoda Rivers'a -- łączenie na podstawie $\hat{y}$